# Markov Chain Monte Carlo

Markov Chain Monte Carlo (MCMC) is another class of techniques to perform interence by sampling directly from the posterior distribution. In this tutorial we will showcase how to use [emcee](https://emcee.readthedocs.io/en/stable/), an *ensemble sampler*. Ensemble samplers uses a set of walkers that move around the parameter space generally spending more time in high probablity regions. Several types of moves are implemented in `emcee` and can be mixed together to form a probabilistic mixture, see [emcee moves documentation](https://emcee.readthedocs.io/en/stable/user/moves/). Accepting or rejecting the move is based on the [Metropolis algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm). 